# Flight Delay Demo - Classical ML with VS Code

## Install prerequisites

Before running the notebook, make sure the correct versions of these libraries are installed.

In [ ]:
!pip install --upgrade azureml-sdk azureml-widgets azureml-train-automl scipy azureml-contrib-notebook azureml-automl-core azureml-explain-model azureml-opendatasets azureml-train-automl-runtime pandas

In [ ]:
!az extension add --name ml

## Setup working directory

The cell below creates our working directory. This will hold our generated scripts.

In [ ]:
import os

project_folder = './scripts'

# Working directory
if not os.path.exists(project_folder):
    os.makedirs(project_folder)

## Write helper files

The `sklearn.preprocessing.LabelEncoder` encodes target labels with value between 0 and n_classes-1.

The `sklearn.model_selection.train_test_split` splits arrays or matrices into random train and test subsets

The `sklearn.metrics.accuracy_score` is an accuracy classification score. In multilabel classification, this function computes subset accuracy: the set of labels predicted for a sample must exactly match the corresponding set of labels in y_true.

The `sklearn.metrics.confusion_matrix` is compute confusion matrix to evaluate the accuracy of a classification.

The `sklearn.metrics.f1_score` computes the F1 score, also known as balanced F-score or F-measure. The F1 score can be interpreted as a weighted average of the precision and recall, where an F1 score reaches its best value at 1 and worst score at 0.

The `sklearn.metrics.precision_score` computes the precision. The precision is the ratio tp / (tp + fp) where tp is the number of true positives and fp the number of false positives.

The `sklearn.metrics.recall_score` computes the recall. The recall is the ratio tp / (tp + fn) where tp is the number of true positives and fn the number of false negatives. The recall is intuitively the ability of the classifier to find all the positive samples.

The `sklearn.metrics.roc_auc_score` computes Area Under the Receiver Operating Characteristic Curve (ROC AUC) from prediction scores.

The `sklearn.metrics.roc_curve` computes Receiver operating characteristic (ROC).

The `Model Class` represents the result of machine learning training. A model is the result of a Azure Machine learning training Run or some other model training process outside of Azure. Regardless of how the model is produced, it can be registered in a workspace, where it is represented by a name and a version. 


For more information on **Model Class**, please visit: [Microsoft Model Class Documentation](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.model.model?view=azure-ml-py)

In [ ]:
%%writefile $project_folder/utils.py
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import joblib
import matplotlib.pyplot as plt
import os
import seaborn as sns
from azureml.core import Model
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, precision_score, recall_score, roc_auc_score, roc_curve
from azureml.core import Model

def split_dataset(X_raw, Y):
    A = X_raw[['UniqueCarrier']]
    X = X_raw.drop(labels=['UniqueCarrier'],axis = 1)
    X = pd.get_dummies(X)


    le = LabelEncoder()
    Y = le.fit_transform(Y)

    X_train, X_test, Y_train, Y_test, A_train, A_test = train_test_split(X_raw, 
                                                        Y, 
                                                        A,
                                                        test_size = 0.2,
                                                        random_state=123,
                                                        stratify=Y)

    # Work around indexing bug
    X_train = X_train.reset_index(drop=True)
    A_train = A_train.reset_index(drop=True)
    X_test = X_test.reset_index(drop=True)
    A_test = A_test.reset_index(drop=True)

    return X_train, X_test, Y_train, Y_test, A_train, A_test 

def prepareDataset(X_raw):
    df = X_raw
    Y = df['ArrDelay15'].values
    synth_df = df.drop(columns=['ArrDelay15'])
    return synth_df, Y

def register_model(name, model, ws):
    print("Registering ", name)
    joblib.dump(value=model, filename="models/flight_delay_weather_.pkl")
    registered_model = Model.register(model_path="models/flight_delay_weather_.pkl",
                                      model_name='flight_delay_weather_',
                                      workspace=ws)
    print("Registered ", registered_model.id)
    return registered_model.id

def analyze_model(clf, X_test, Y_test, preds):
    accuracy = accuracy_score(Y_test, preds)
    print(f'Accuracy', np.float(accuracy))

    precision = precision_score(Y_test, preds, average="macro")
    print(f'Precision', np.float(precision))

    recall = recall_score(Y_test, preds, average="macro")
    print(f'Recall', np.float(recall))

    f1score = f1_score(Y_test, preds, average="macro")
    print(f'F1 Score', np.float(f1score))

    class_names = clf.classes_
    fig, ax = plt.subplots()
    tick_marks = np.arange(len(class_names))
    plt.xticks(tick_marks, class_names)
    plt.yticks(tick_marks, class_names)
    sns.heatmap(pd.DataFrame(confusion_matrix(Y_test, preds)), annot=True, cmap='YlGnBu', fmt='g')
    ax.xaxis.set_label_position('top')
    plt.tight_layout()
    plt.title('Confusion Matrix', y=1.1)
    plt.ylabel('Actual label')
    plt.xlabel('Predicted label')
    plt.show()
    plt.close()

    preds_proba = clf.predict_proba(X_test)[::,1]
    fpr, tpr, _ = roc_curve(Y_test, preds_proba, pos_label = clf.classes_[1])
    auc = roc_auc_score(Y_test, preds_proba)
    plt.plot(fpr, tpr, label="data 1, auc=" + str(auc))
    plt.legend(loc=4)
    plt.show()
    plt.close()

In [ ]:
import warnings
warnings.filterwarnings("ignore")

## Connect to Workspace

In the next cell, we create a new Workspace config object using the `<subscription_id>`, `<resource_group_name>`, and `<workspace_name>`. This will fetch the matching Workspace and prompt you for authentication. Please click on the link and input the provided details.

For more information on **Workspace**, please visit: [Microsoft Workspace Documentation](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.workspace.workspace?view=azure-ml-py)

`<subscription_id>` = You can get this ID from the landing page of your Resource Group.

`<resource_group_name>` = This is the name of your Resource Group.

`<workspace_name>` = This is the name of your Workspace.

In [ ]:
from azureml.core.workspace import Workspace

try:
    # Get instance of the Workspace and write it to config file
    ws = Workspace(
        subscription_id = '<subscription_id>', 
        resource_group = '<resource_group>', 
        workspace_name = '<workspace_name>')

    # Writes workspace config file
    ws.write_config()
    
    print('Library configuration succeeded')
except Exception as e:
    print(e)
    print('Workspace not found')

## Load Data

Pandas `read_csv()` function is used to read local file which containes our data set.

In [ ]:
import pandas as pd

data = pd.read_csv('./flightdelayweather_ds_clean.csv')

# Data exploration and visualization

## Observe training dataset

`pandas.DataFrame.describe()` function generates descriptive statistics.

`pandas.DataFrame.head(n=5)` function returns the frist *n* rows.

In [ ]:
data.describe()

In [ ]:
data.head(5)

## Plot training data

Let's take a look at the distribuition plot for the distance of the flights in our training dataset. We will be using `seaborn` as our thirdparty library that will help us plot the data.

`seaborn.set()` function is used to set *theme* parameters.

`seabor.displot()` function provides access to several approaches for visualizing the univariate or bivariate distribution of data, including subsets of data defined by semantic mapping and faceting across multiple subplots.


In [ ]:
import seaborn as sns

sns.set(color_codes='True')
col_list = ['Distance']
df_col = data[col_list]
# sns.distplot(df_col)
sns.displot(df_col)

The following pair plot shows the relationship between Distance, CRSDepTime and CRSArrTime.

`seaborn.pairplot` plots pairwise relationships in a dataset.

In [ ]:
col_list = ['Distance', 'CRSDepTime', 'CRSArrTime']
df_2 = data[col_list]
sns.pairplot(df_2)

# Machine Learning

The `Pipeline()` function purpose is to assemble several steps that can be cross-validated together while setting different parameters.

The `sklearn.linear_model.LogisticRegression` class implements regularized logistic regression using the ‘liblinear’ library, ‘newton-cg’, ‘sag’, ‘saga’ and ‘lbfgs’ solvers.

The `sklearn.preprocessing.StandardScaler()` function standardizes features by removing the mean and scaling to unit variance.

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from scripts.utils import *
import joblib
import numpy as np
import pandas as pd

# Fetch dataset from the run by name
synth_df, Y = prepareDataset(data)

#Split dataset
X_train, X_test, Y_train, Y_test, A_train, A_test = split_dataset(synth_df, Y)

# Setup scikit-learn pipeline
numeric_transformer = Pipeline(steps=[('scaler', StandardScaler())])

clf = Pipeline(steps=[('classifier', LogisticRegression(solver='liblinear', fit_intercept=True))])

model = clf.fit(X_train, Y_train)
preds = clf.predict(X_test)
analyze_model(clf, X_test, Y_test, preds)

## Register Model

Next, register the model obtained from the best run. In order to register the model, the function `register()` should be called. This will take care of registering the model obtained from the best run.

The Model.register() function registers a model with the provided workspace.

For more information on **Model Class**, please visit: [Microsoft Model Class Documentation](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.model.model?view=azure-ml-py)

In [ ]:
os.makedirs('models', exist_ok=True)

joblib.dump(value=model, filename="models/flight_delay_weather_.pkl")
registered_model = Model.register(model_path="models/flight_delay_weather_.pkl",
                                  model_name='flight_delay_weather_',
                                  workspace=ws)
print("Registered ", registered_model.id)

## Fetch the model from the registry

Retrieve a reference to the model we just uploaded.

In [ ]:
from azureml.core.model import Model
from scripts.utils import *

synth_df, Y = prepareDataset(data)
X_train, X_test, Y_train, Y_test, A_train, A_test = split_dataset(synth_df, Y)
model = Model(ws, 'flight_delay_weather_')
model.version

# Deployment

## Create managed-endpoints directory

Create a new directory to hold the configuration files for deploying a managed endpoint.

In [ ]:
import os

managed_endpoints = './managed-endpoints'

# Working directory
if not os.path.exists(managed_endpoints):
    os.makedirs(managed_endpoints)
    
if os.path.exists(os.path.join(managed_endpoints,".amlignore")):
  os.remove(os.path.join(managed_endpoints,".amlignore"))

## Create Scoring File

Creating the scoring file is next step before deploying the service. This file is responsible for the actual generation of predictions using the model. The values or scores generated can represent predictions of future values, but they might also represent a likely category or outcome.

The first thing to do in the scoring file is to fetch the model. This is done by calling `Model.get_model_path()` and passing the model name as a parameter.

After the model has been loaded, the function `model.predict()` function should be called to start the scoring process.

For more information on **Machine Learning - Score**, please visit: [Microsoft Machine Learning - Score Documentation](https://docs.microsoft.com/en-us/azure/machine-learning/studio-module-reference/machine-learning-score)


In [ ]:
%%writefile $managed_endpoints/score.py
import os
import json
import time
import numpy as np
import pandas as pd
import azureml.automl.core
from sklearn.externals import joblib
 
def init():
    global model
    print ("model initialized" + time.strftime("%H:%M:%S"))
    
    # this name is model.id of model that we want to deploy
    model_path = os.path.join(os.getenv("AZUREML_MODEL_DIR"), "flight_delay_weather_.pkl")
    
    # deserialize the model file back into a sklearn model
    model = joblib.load(model_path)
    
def run(data):
    try:
        data = json.loads(data)
        df = pd.DataFrame(data['data'], columns=['Month', 'DayofMonth', 'DayOfWeek', 'CRSDepTime', 'CRSArrTime',
       'UniqueCarrier', 'CRSElapsedTime', 'Origin', 'Dest', 'Distance',
       'Origin_Lat', 'Origin_Lon', 'Origin_State', 'Dest_Lat', 'Dest_Lon',
       'Dest_State', 'Origin_dayl', 'Dest_dayl', 'Origin_prcp', 'Dest_prcp',
       'Origin_srad', 'Dest_srad', 'Origin_swe', 'Dest_swe', 'Origin_tmax',
       'Dest_tmax', 'Origin_tmin', 'Dest_tmin', 'Origin_vp', 'Dest_vp']) 
        result = model.predict(df)
    except Exception as e:
        result = str(e)
        print(result)
        return {"error": result}
    return {"result":result.tolist()}

## Create the environment definition

The following file contains the details of the environment to host the model and code. 

In [ ]:
%%writefile $managed_endpoints/score-new.yml
name: fd-local-mng-env
channels:
  - conda-forge
dependencies:
  - python=3.7
  - numpy
  - pip
  - scikit-learn==0.22.1
  - scipy
  - pip:
    - azureml-defaults
    - azureml-sdk[notebooks,automl]
    - pandas
    - inference-schema[numpy-support]
    - joblib
    - numpy
    - scipy

## Define the endpoint configuration
Specific inputs are required to deploy a model on an online endpoint:

1. Model files.
1. The code that's required to score the model.
1. An environment in which your model runs.
1. Settings to specify the instance type and scaling capacity.

In [ ]:
%%writefile $managed_endpoints/endpointconfig.yml
name: fd-local-mng-endpoint
type: online
auth_mode: key
traffic:
  blue: 100

deployments:
  #blue deployment
  - name: blue
    model: azureml:flight_delay_weather_:1
    code_configuration:
      code:
        local_path: ./
      scoring_script: score.py
    environment: 
      name: fd-local-mng-env
      version: 1
      path: ./
      conda_file: file:./score-new.yml
      docker:
          image: mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04:20210727.v1
    instance_type: Standard_DS3_v2
    scale_settings:
      scale_type: manual
      instance_count: 1
      min_instances: 1
      max_instances: 2

## Deploy your managed online endpoint to Azure

This deployment might take up to 15 minutes, depending on whether the underlying environment or image is being built for the first time. Subsequent deployments that use the same environment will finish processing more quickly.

In [ ]:
!az ml endpoint create -g [your resource group name] -w [your AML workspace name] -n fd-local-mng-endpoint -f ./managed-endpoints/endpointconfig.yml

## Generate a sample request JSON file

Export some test data to a JSON file we can send to the endpoint.

In [ ]:
%%writefile $managed_endpoints/sample-request.json
{"data": [
[6.0,21.0,6.0,1330.0,1600.0,9.0,150.0,16.0,93.0,745.0,33.64044444,-84.42694444,8.0,40.69249722,-74.16866056,29.0,51148.8,53568.0,2.0,0.0,438.4,451.2,0.0,0.0,30.5,28.5,18.0,15.0,2040.0,1720.0],
[4.0,2.0,3.0,1910.0,2035.0,11.0,85.0,222.0,62.0,361.0,35.87763889,-78.78747222,25.0,39.99798528,-82.89188278,33.0,44928.0,45273.6,0.0,0.0,355.2,438.4,0.0,0.0,23.0,12.5,12.0,1.5,1400.0,680.0],
[1.0,3.0,4.0,935.0,1224.0,16.0,229.0,207.0,78.0,1302.0,39.87195278,-75.24114083,36.0,32.89595056,-97.0372,41.0,33177.6,35596.8,0.0,0.0,156.8,252.8,0.0,0.0,-2.0,6.5,-8.0,-4.5,320.0,440.0],
[4.0,3.0,4.0,1000.0,1252.0,16.0,172.0,207.0,206.0,951.0,39.87195278,-75.24114083,36.0,26.68316194,-80.09559417,7.0,45273.6,44582.4,0.0,4.0,425.6,220.8,0.0,0.0,12.0,28.0,0.5,22.5,640.0,2720.0],
[1.0,21.0,1.0,800.0,1045.0,15.0,105.0,198.0,129.0,589.0,41.979595,-87.90446417,12.0,38.94453194,-77.45580972,43.0,33868.8,34905.6,2.0,0.0,256.0,246.4,56.0,0.0,-7.0,-3.0,-17.0,-13.5,160.0,200.0],
[3.0,12.0,3.0,1640.0,1952.0,5.0,192.0,89.0,101.0,1065.0,40.69249722,-74.16866056,29.0,26.07258333,-80.15275,7.0,41817.6,42508.8,0.0,0.0,336.0,368.0,0.0,0.0,10.0,27.0,0.5,19.5,640.0,2280.0],
[3.0,19.0,3.0,1229.0,1346.0,6.0,77.0,151.0,76.0,214.0,40.77724306,-73.87260917,32.0,38.85208333,-77.03772222,43.0,42854.4,42854.4,22.0,0.0,204.8,307.2,0.0,0.0,10.0,15.0,4.0,6.5,800.0,960.0],
[4.0,18.0,5.0,1210.0,1503.0,4.0,173.0,139.0,169.0,944.0,40.63975111,-73.77892556,32.0,28.42888889,-81.31602778,7.0,47692.8,45964.8,0.0,0.0,524.8,508.8,0.0,0.0,22.5,26.0,5.5,11.5,920.0,1360.0],
[11.0,1.0,6.0,615.0,745.0,9.0,90.0,130.0,18.0,432.0,39.71732917,-86.29438417,13.0,33.64044444,-84.42694444,8.0,36633.6,38016.0,0.0,0.0,297.6,387.2,0.0,0.0,22.0,20.5,5.0,2.0,880.0,720.0],
[11.0,24.0,1.0,936.0,1123.0,8.0,107.0,208.0,77.0,602.0,33.43416667,-112.00805559999999,3.0,39.85840806,-104.6670019,5.0,35942.4,34214.4,0.0,0.0,297.6,291.2,0.0,0.0,27.5,17.0,10.0,-9.5,520.0,280.0]]}

## Invoke the endpoint to score data by using your model

You can use either the invoke command or a REST client of your choice to invoke the endpoint and score against it.


In [ ]:
!az ml endpoint invoke -g [your resource group name] -w [your AML workspace name] -n fd-local-mng-endpoint --request-file ./managed-endpoints/sample-request.json

## Optional: Create/connect to the Kubernetes compute cluster 

The `AksCompute Class` manages an Azure Kubernetes Service compute target in Azure Machine Learning.

The `ComputeTarget Class` is an abstract parent class for all compute targets managed by Azure Machine Learning. A compute target is a designated compute resource/environment where you run your training script or host your service deployment. 

The `ComputeTargetException` is an exception related to failures when creating, interacting with, or configuring a compute target. This exception is commonly raised for failures attaching a compute target, missing headers, and unsupported configuration values.

For more information on **AksCompute Class**, please visit: [Microsoft Machine Learning - AksCompute Class Documentation](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.compute.aks.akscompute?view=azure-ml-py)

For more information on **ComputeTarget Class**, please visit: [Microsoft Machine Learning - ComputeTarget Class Documentation](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.compute.computetarget?view=azure-ml-py)

For more information on **ComputeTargetException Class**, please visit: [Microsoft Machine Learning - ComputeTargetException Class Documentation](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.exceptions.computetargetexception?view=azure-ml-py)


In [ ]:
from azureml.core.compute import AksCompute
from azureml.core.compute import ComputeTarget
from azureml.exceptions import ComputeTargetException

prov_config = AksCompute.provisioning_configuration(location='westus2')

try:
    aks_target = AksCompute(ws, 'flight-delay-aks')
except ComputeTargetException:
    # Create the cluster
    aks_target = ComputeTarget.create(workspace = ws, 
                            name = 'flight-delay-aks', 
                            provisioning_configuration = prov_config)
    aks_target.wait_for_completion(True)

## Optional: Create Scoring File

Creating the scoring file is next step before deploying the service. This file is responsible for the actual generation of predictions using the model. The values or scores generated can represent predictions of future values, but they might also represent a likely category or outcome.

The first thing to do in the scoring file is to fetch the model. This is done by calling `Model.get_model_path()` and passing the model name as a parameter.

After the model has been loaded, the function `model.predict()` function should be called to start the scoring process.

For more information on **Machine Learning - Score**, please visit: [Microsoft Machine Learning - Score Documentation](https://docs.microsoft.com/en-us/azure/machine-learning/studio-module-reference/machine-learning-score)

In [ ]:
%%writefile score.py
import json
import time
import numpy as np
import pandas as pd
from azureml.core.model import Model
from sklearn.externals import joblib
 
def init():
    global model
    print ("model initialized" + time.strftime("%H:%M:%S"))
    
    # this name is model.id of model that we want to deploy
    model_path = Model.get_model_path(model_name = 'flight_delay_weather_')
    
    # deserialize the model file back into a sklearn model
    model = joblib.load(model_path)
    
def run(data):
    try:
        data = json.loads(data)
        df = pd.DataFrame(data['data'], columns=['Month', 'DayofMonth', 'DayOfWeek', 'CRSDepTime', 'CRSArrTime',
       'UniqueCarrier', 'CRSElapsedTime', 'Origin', 'Dest', 'Distance',
       'Origin_Lat', 'Origin_Lon', 'Origin_State', 'Dest_Lat', 'Dest_Lon',
       'Dest_State', 'Origin_dayl', 'Dest_dayl', 'Origin_prcp', 'Dest_prcp',
       'Origin_srad', 'Dest_srad', 'Origin_swe', 'Dest_swe', 'Origin_tmax',
       'Dest_tmax', 'Origin_tmin', 'Dest_tmin', 'Origin_vp', 'Dest_vp']) 
        result = model.predict(df)
    except Exception as e:
        result = str(e)
        print(result)
        return {"error": result}
    return {"result":result.tolist()}

## Optional: Deploy the model to Kubernetes

Now that the AKS cluster has been deployed , it’s time to create an `InferenceConfig` object by calling its constructor and passing the runtime type, the path to the `entry_script` (score.py), and the `conda_file` (the file that holds the environment dependencies).

Next, define the configuration of the web service to deploy. This is done by calling `AksWebservice.deploy_configuration()` and passing along the number of `cpu_cores` and `memory_gb` that the service needs.

Finally, in order to deploy the model and service to the created AKS cluster, the function `Model.deploy()` should be called, passing along the workspace object, a list of models to deploy, the defined inference configuration, deployment configuration, and the AKS object created in the step above.

For more information on **InferenceConfig**, please visit: [Microsoft InferenceConfig Documentation](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.model.inferenceconfig?view=azure-ml-py)

For more information on **AksWebService**, please visit: [Microsoft AksWebService Documentation](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.webservice.akswebservice?view=azure-ml-py)

For more information on **Model**, please visit: [Microsoft Model Documentation](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.model.model?view=azure-ml-py)


**Note:** Please wait for the execution of the cell to finish before moving forward.

In [ ]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AksWebservice
from azureml.exceptions import WebserviceException

inference_config = InferenceConfig(runtime= "python",
                                    entry_script="score.py",
                                    conda_file="score-new.yml")

deployment_config = AksWebservice.deploy_configuration(cpu_cores = 1, 
                                                        memory_gb = 1)

try:
    service = AksWebservice(ws, 'flight-delay-local')
    print(service.state)
except WebserviceException:
    service = model.deploy(ws, 
                            'flight-delay-local', 
                            [model], 
                            inference_config, 
                            deployment_config, 
                            aks_target
                            )

    service.wait_for_deployment(show_output = True)
    print(service.state)

## Optional: Connect to the deployed webservice

Now with test data, we can get it into a suitable format to consume the web service. First an instance of the web service should be obtained by calling the constructor `Webservice()` with the Workspace object and the service name as parameters. Sanitizing of the data is then performed in order to avoid sending unexpected columns to the web service. Finally, call the service via POST using the `requests` module. `requests.post()` will call the deployed web service. It takes for parameters the service URL, the test data, and a headers dictionary that contains the authentication token.

For more information on **Webservice**, please visit: [Microsoft Webservice Documentation](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.webservice?view=azure-ml-py)

In [ ]:
import json
import csv
import requests
import pandas as pd
from azureml.core.webservice import Webservice

aks_service = Webservice(ws, 'flight-delay-local')
aks_service.update(enable_app_insights=True)

# prepare the test data
sample = X_test.sample(n=10, random_state=4).values.tolist()

headers = {'Content-Type':'application/json'}

if aks_service.auth_enabled:
    headers['Authorization'] = 'Bearer '+ aks_service.get_keys()[0]

output_df = []
for x in sample:
    test_sample = json.dumps({'data': [x]})
    response = requests.post(aks_service.scoring_uri, data=test_sample, headers=headers)
    prediction = [response.json()['result'][0]]
    prediction.extend(x)
    output_df.append(prediction)

## Optional: Present scoring service predictions

Let's format our service responses and present them in a suitable way to our end users.

In [ ]:
def highlight_delays(val):
    return 'background-color: yellow' if val == True else ''

predictions = pd.DataFrame(output_df, columns =['Prediction', 'Month', 'DayofMonth', 'DayOfWeek', 'CRSDepTime', 'CRSArrTime', 'UniqueCarrier', 'CRSElapsedTime', 'Origin', 'Dest', 'Distance', 'Origin_Lat', 'Origin_Lon', 'Origin_State', 'Dest_Lat', 'Dest_Lon', 'Dest_State', 'Origin_dayl', 'Dest_dayl', 'Origin_prcp', 'Dest_prcp', 'Origin_srad', 'Dest_srad', 'Origin_swe', 'Dest_swe', 'Origin_tmax', 'Dest_tmax', 'Origin_tmin', 'Dest_tmin', 'Origin_vp', 'Dest_vp'])
predictions = predictions.style.applymap(highlight_delays, subset=['Prediction'])
predictions